Imported the necessary Libraries

In [12]:
import cv2
import numpy as np
import os
from google.colab import drive


Mounted the Google drive where the images are located

In [13]:
# Mount the Google Drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


Unzipped the dataset

In [14]:
!cp /gdrive/MyDrive/dataset.zip .
!unzip dataset.zip

Archive:  dataset.zip
replace dataset/test/image_0009.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# **Method 1 : Custom method for FisherFaces**

Loaded the 19 person images and preprocessed them then converted them to numpy array

In [15]:
# Set the dimensions for resizing the images
image_width, image_height = 100, 100

# Load the images from the train folder
train_images = []
train_labels = []

person_label = 0  # Define the person label variable

for person_folder in os.listdir("dataset/train"):
    person_folder_path = os.path.join("dataset/train", person_folder)
    if os.path.isdir(person_folder_path):
        for image_file in os.listdir(person_folder_path):
            image_path = os.path.join(person_folder_path, image_file)
            if os.path.isfile(image_path):
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, (image_width, image_height))
                image = image.astype(np.float32) / 255.0
                train_images.append(image)
                train_labels.append(person_label)
        person_label += 1  # Increment the person label for the next folder

# Convert the list of images and labels to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

Calculated the average face and subtracted the averege face from all the faces

In [16]:

# Compute the mean image
mean_image = np.mean(train_images, axis=0)

# Compute the deviation of each image from the mean image
deviations = train_images - mean_image
deviations = deviations.reshape(deviations.shape[0], -1)


Calculated the covariance matrix and sorted the eigenvectors by their valuesthen selected the top 10 eigen vectors

In [18]:
# Compute the covariance matrix using SVD
U, S, Vt = np.linalg.svd(deviations, full_matrices=False)
eigenvalues = S ** 2 / (deviations.shape[0] - 1)
eigenvectors = Vt.T

# Sort the eigenvectors based on their corresponding eigenvalues
sorted_indices = np.argsort(eigenvalues)[::-1]
sorted_eigenvectors = eigenvectors[:, sorted_indices]

selected_eigenvectors = sorted_eigenvectors[:, :10]



Projected the feature vectors onto the selected vectors then created the custom recognizer class which has train and predict method

In [19]:
# Compute the feature vectors by projecting the deviations onto the selected eigenvectors
feature_vectors = np.dot(deviations, selected_eigenvectors)

# Create the Fisherface recognizer
class FisherfaceRecognizer:
    def __init__(self):
        self.mean_image = None
        self.selected_eigenvectors = None
        self.train_labels = None
        self.feature_vectors = None

    def train(self, train_images, train_labels):
        self.mean_image = np.mean(train_images, axis=0)
        deviations = train_images - self.mean_image
        deviations = deviations.reshape(deviations.shape[0], -1)  # Reshape deviations to 2D
        U, S, Vt = np.linalg.svd(deviations, full_matrices=False)
        eigenvalues = S ** 2 / (deviations.shape[0] - 1)
        eigenvectors = Vt.T
        sorted_indices = np.argsort(eigenvalues)[::-1]
        sorted_eigenvectors = eigenvectors[:, sorted_indices]
        k = 10  # Choose the desired dimensionality of the feature space
        self.selected_eigenvectors = sorted_eigenvectors[:, :k]
        self.train_labels = train_labels
        self.feature_vectors = np.dot(deviations, self.selected_eigenvectors)

    def predict(self, test_image):
        test_image = test_image.astype(np.float32) / 255.0
        test_image = cv2.resize(test_image, (image_width, image_height))
        test_image = test_image.flatten() - self.mean_image.flatten()
        test_feature_vector = np.dot(test_image, self.selected_eigenvectors)
        distances = np.linalg.norm(self.feature_vectors - test_feature_vector, axis=1)
        nearest_index = np.argmin(distances)
        predicted_label = self.train_labels[nearest_index]
        confidence = 1.0 / distances[nearest_index]
        return predicted_label, confidence


Initialized the recognizer and trained it with the traing images

In [20]:

# Create an instance of the custom Fisherface recognizer
fisherface = FisherfaceRecognizer()

# Train the Fisherface recognizer
fisherface.train(train_images, train_labels)

Loaded and preprocessed the images to prepare them to be predicted

In [21]:
# Load the images from the test folder
test_images = []

for image_file in os.listdir("dataset/test"):
    image_path = os.path.join("dataset/test", image_file)
    if os.path.isfile(image_path):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:  # Check if the image was successfully read
            image = cv2.resize(image, (image_width, image_height))
            image = image.astype(np.float32) / 255.0
            test_images.append(image)

# Convert the list of test images to a numpy array
test_images = np.array(test_images)

print(test_images)


[[[0.09019608 0.07058824 0.08235294 ... 0.3254902  0.29803923 0.29803923]
  [0.07843138 0.09411765 0.08627451 ... 0.2901961  0.3019608  0.29411766]
  [0.09803922 0.06666667 0.10588235 ... 0.2901961  0.3019608  0.2901961 ]
  ...
  [0.3372549  0.33333334 0.33333334 ... 0.29411766 0.31764707 0.2901961 ]
  [0.3372549  0.3372549  0.36078432 ... 0.29803923 0.3019608  0.29411766]
  [0.34509805 0.33333334 0.3372549  ... 0.30588236 0.3019608  0.29411766]]

 [[0.20392157 0.3372549  0.34901962 ... 0.42352942 0.4392157  0.41568628]
  [0.16078432 0.34117648 0.35686275 ... 0.44313726 0.42745098 0.43529412]
  [0.22745098 0.32941177 0.3529412  ... 0.4392157  0.44313726 0.43529412]
  ...
  [0.42745098 0.40784314 0.43529412 ... 0.5137255  0.5058824  0.5137255 ]
  [0.4        0.42352942 0.43137255 ... 0.5137255  0.5137255  0.5019608 ]
  [0.42352942 0.44313726 0.45490196 ... 0.5058824  0.5019608  0.49411765]]

 [[0.41960785 0.40784314 0.40784314 ... 0.7137255  0.7176471  0.69803923]
  [0.41960785 0.427450

Perform labeling and print out the result.

In [22]:

# Perform face recognition on the test images
predicted_labels = []
confidences = []
i=1

for test_image in test_images:
    # Preprocess the test image
    test_image = test_image.astype(np.float32) / 255.0
    test_image = cv2.resize(test_image, (image_width, image_height))

    # Compute the Fisherface projection of the test image
    label, confidence = fisherface.predict(test_image)

    print(f"The predicted label for the test image {i} is {label} with confidence {confidence}")
    i+=1
    # Append the predicted label and confidence to the lists
    predicted_labels.append(label)
    confidences.append(confidence)

The predicted label for the test image 1 is 10 with confidence 0.04040177374242267
The predicted label for the test image 2 is 10 with confidence 0.040402016586860004
The predicted label for the test image 3 is 10 with confidence 0.04040190761784115
The predicted label for the test image 4 is 10 with confidence 0.04040206328804803
The predicted label for the test image 5 is 10 with confidence 0.040402001019821325
The predicted label for the test image 6 is 10 with confidence 0.040401879597331304
The predicted label for the test image 7 is 10 with confidence 0.04040196054557689
The predicted label for the test image 8 is 10 with confidence 0.040401820443049276
The predicted label for the test image 9 is 10 with confidence 0.04040211932961617
The predicted label for the test image 10 is 10 with confidence 0.04040194497858139
The predicted label for the test image 11 is 10 with confidence 0.04040188582410791
The predicted label for the test image 12 is 10 with confidence 0.040401761288940

# **Method 2 : Used the builtin method for FisherFaces**

Load the training dataset then Convert to numpy array

In [23]:
# Set the dimensions for resizing the images
image_width, image_height = 100, 100

# Load the images from the train folder
train_images = []
train_labels = []

person_label = 0

for person_folder in os.listdir("dataset/train"):
    person_folder_path = os.path.join("dataset/train", person_folder)
    if os.path.isdir(person_folder_path):
        for image_file in os.listdir(person_folder_path):
            image_path = os.path.join(person_folder_path, image_file)
            if os.path.isfile(image_path):
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, (image_width, image_height))
                image = image.astype(np.float32) / 255.0
                train_images.append(image)
                train_labels.append(person_label)
        person_label += 1

print(person_label)
# Convert the list of images and labels to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
print(train_images)
print(train_labels)

19
[[[0.7294118  0.7372549  0.74509805 ... 0.64705884 0.6392157  0.6392157 ]
  [0.7411765  0.74509805 0.7529412  ... 0.65882355 0.63529414 0.6392157 ]
  [0.74509805 0.73333335 0.7411765  ... 0.6431373  0.6392157  0.63529414]
  ...
  [0.6745098  0.68235296 0.68235296 ... 0.6431373  0.63529414 0.62352943]
  [0.67058825 0.6745098  0.6784314  ... 0.6431373  0.6392157  0.6313726 ]
  [0.6627451  0.6745098  0.6862745  ... 0.627451   0.627451   0.61960787]]

 [[0.6509804  0.5176471  0.45882353 ... 0.6784314  0.67058825 0.6627451 ]
  [0.6509804  0.5529412  0.44313726 ... 0.68235296 0.65882355 0.6627451 ]
  [0.63529414 0.5137255  0.44705883 ... 0.6862745  0.6745098  0.6666667 ]
  ...
  [0.03137255 0.01568628 0.03921569 ... 0.04705882 0.05098039 0.3019608 ]
  [0.04313726 0.02745098 0.02352941 ... 0.06666667 0.04705882 0.29803923]
  [0.03529412 0.05098039 0.05882353 ... 0.05490196 0.01176471 0.3254902 ]]

 [[0.2509804  0.2627451  0.27058825 ... 0.16862746 0.11764706 0.41568628]
  [0.25882354 0.250

Initialized the Fisherface recognizer and trained it with the training dataset.

In [24]:
# Create the Fisherface recognizer
fisherface = cv2.face.FisherFaceRecognizer_create()

# Train the Fisherface recognizer
fisherface.train(train_images, train_labels)


Loaded the test images, preprocessed them and convet them to numpy array.

In [30]:

# Load the images from the test folder
test_images = []

for image_file in os.listdir("dataset/test"):
    image_path = os.path.join("dataset/test", image_file)
    if os.path.isfile(image_path):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:  # Check if the image was successfully read
            image = cv2.resize(image, (image_width, image_height))
            image = image.astype(np.float32) / 255.0
            test_images.append(image)

# Convert the list of test images to a numpy array
test_images = np.array(test_images)

print(test_images)

[[[0.09019608 0.07058824 0.08235294 ... 0.3254902  0.29803923 0.29803923]
  [0.07843138 0.09411765 0.08627451 ... 0.2901961  0.3019608  0.29411766]
  [0.09803922 0.06666667 0.10588235 ... 0.2901961  0.3019608  0.2901961 ]
  ...
  [0.3372549  0.33333334 0.33333334 ... 0.29411766 0.31764707 0.2901961 ]
  [0.3372549  0.3372549  0.36078432 ... 0.29803923 0.3019608  0.29411766]
  [0.34509805 0.33333334 0.3372549  ... 0.30588236 0.3019608  0.29411766]]

 [[0.20392157 0.3372549  0.34901962 ... 0.42352942 0.4392157  0.41568628]
  [0.16078432 0.34117648 0.35686275 ... 0.44313726 0.42745098 0.43529412]
  [0.22745098 0.32941177 0.3529412  ... 0.4392157  0.44313726 0.43529412]
  ...
  [0.42745098 0.40784314 0.43529412 ... 0.5137255  0.5058824  0.5137255 ]
  [0.4        0.42352942 0.43137255 ... 0.5137255  0.5137255  0.5019608 ]
  [0.42352942 0.44313726 0.45490196 ... 0.5058824  0.5019608  0.49411765]]

 [[0.41960785 0.40784314 0.40784314 ... 0.7137255  0.7176471  0.69803923]
  [0.41960785 0.427450

Predict the lables of each test images and return their results

In [31]:

# Perform face recognition on the test images
predicted_labels = []
confidences = []
i=0
for test_image in test_images:
    # print(test_image)
    # Compute the Fisherface projection of the test image
    label, confidence = fisherface.predict(test_image)

    print(f"The predicted label for test image {i+1} is {label} with confidence {confidence}")
    i+=1
    # Append the predicted label and confidence to the lists
    predicted_labels.append(label)
    confidences.append(confidence)

The predicted label for test image 1 is 10 with confidence 2.942938288533638
The predicted label for test image 2 is 14 with confidence 3.252244319865345
The predicted label for test image 3 is 11 with confidence 4.6267932728828045
The predicted label for test image 4 is 9 with confidence 2.450743964563048
The predicted label for test image 5 is 0 with confidence 2.844033764880331
The predicted label for test image 6 is 11 with confidence 3.902083586038028
The predicted label for test image 7 is 15 with confidence 2.9093983521989695
The predicted label for test image 8 is 10 with confidence 3.610911657698033
The predicted label for test image 9 is 4 with confidence 3.163272823073784
The predicted label for test image 10 is 9 with confidence 3.4104404722547463
The predicted label for test image 11 is 3 with confidence 3.9576516605760976
The predicted label for test image 12 is 3 with confidence 4.1494432377075885
The predicted label for test image 13 is 10 with confidence 3.555316990687